In [1]:
import pandas as pd
from scipy import signal
import numpy as np
import pywt
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,f1_score, classification_report, confusion_matrix , accuracy_score, precision_score, recall_score, f1_score, roc_curve ,roc_auc_score,ConfusionMatrixDisplay
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,f1_score, classification_report, confusion_matrix , accuracy_score, precision_score, recall_score, f1_score, roc_curve ,roc_auc_score
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold 

# Class 0 Hand at Rest

In [2]:
df = pd.read_csv('D:/TY Sem 2/EDI/15k/class.1.csv')
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002
1,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002
2,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002
3,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002
4,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002
...,...,...,...,...,...,...,...,...
14995,0.00002,0.00006,0.00017,0.00002,-0.00001,0.00001,0.00001,-0.00001
14996,0.00002,0.00006,0.00017,0.00002,-0.00001,0.00001,0.00001,-0.00001
14997,0.00002,0.00006,0.00017,0.00002,-0.00001,0.00001,0.00001,-0.00001
14998,0.00002,0.00006,0.00017,0.00002,-0.00001,0.00001,0.00001,-0.00001


In [3]:
# Define the notch frequency and Q-factor
f0 = 60  # Notch frequency (Hz)
Q = 10   # Q-factor
# Define the filter parameters
fs = 801       # Sampling frequency (Hz)
lowcut = 20    # Low cutoff frequency (Hz)
highcut = 400  # High cutoff frequency (Hz)
order = 4      # Filter order


In [4]:
# Apply the filters
for col in df.columns:
    # Get the EMG signal from the current column
    emg = df[col].values
    
    # Apply the Butterworth filter
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    emg_filtered = signal.filtfilt(b, a, emg)
    
    # Apply the notch filter
    wo = f0 / (fs / 2)  # Normalized frequency
    bw = wo / Q
    b_notch, a_notch = signal.iirnotch(wo, bw)
    emg_filtered_notched = signal.filtfilt(b_notch, a_notch, emg_filtered)
    
    # Replace the original signal with the filtered signal
    df[col] = emg_filtered_notched

In [5]:
emg_filtered_notched

array([ 8.74776153e-08, -4.02690680e-07, -9.89269034e-07, ...,
        1.86589391e-06,  1.20375281e-06,  1.13471039e-06])

In [6]:
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,-0.000001,-1.799053e-07,-9.170762e-07,3.461078e-08,-1.159242e-07,1.779539e-07,1.187128e-08,8.747762e-08
1,-0.000001,-1.663784e-06,-2.745208e-06,-2.863672e-06,-1.461575e-06,-3.630988e-07,-1.731078e-07,-4.026907e-07
2,-0.000001,-3.535473e-06,-3.935511e-06,-5.788935e-06,-2.463227e-06,-9.493861e-07,-2.841848e-07,-9.892690e-07
3,-0.000001,-5.009764e-06,-5.677528e-06,-8.674837e-06,-3.792298e-06,-1.498914e-06,-4.636945e-07,-1.477938e-06
4,-0.000001,-6.865404e-06,-6.755049e-06,-1.157189e-05,-4.768351e-06,-2.092187e-06,-5.636133e-07,-2.064351e-06
...,...,...,...,...,...,...,...,...
14995,0.000010,2.511745e-05,3.443233e-05,1.516562e-05,2.555254e-06,1.029263e-05,4.015418e-06,2.634456e-06
14996,0.000008,2.017750e-05,2.421797e-05,8.813513e-06,2.726829e-06,7.957005e-06,2.840759e-06,1.954942e-06
14997,0.000006,1.172591e-05,1.619861e-05,6.103692e-06,6.971127e-07,5.253553e-06,2.179057e-06,1.865894e-06
14998,0.000004,6.378957e-06,5.355469e-06,-8.862703e-07,8.038092e-07,2.765304e-06,9.886332e-07,1.203753e-06


In [7]:
# Set the window size and overlap
window_size = 100   # number of samples in each window
overlap = 50       # number of samples to overlap between windows

In [8]:
# Get the number of rows and columns in the DataFrame
num_rows, num_cols = df.shape

In [9]:
# Initialize a list to store the sliced data
sliced_data = []
for col in range(num_cols):
    # Get the data for the current channel
    channel_data = df.iloc[:, col].values
    
    # Compute the number of windows
    num_windows = int(np.ceil((num_rows - window_size) / overlap)) + 1
    
    # Loop over each window
    for i in range(num_windows):
        # Compute the start and end indices of the window
        start_index = i * overlap
        end_index = start_index + window_size
        
        # Make sure the end index does not exceed the number of rows
        if end_index > num_rows:
            end_index = num_rows
        
        # Get the data for the current window
        window_data = channel_data[start_index:end_index]
        
        # Add the window data to the sliced data list
        sliced_data.append(window_data)

In [10]:
# Convert the sliced data list to a numpy array
sliced_data = np.array(sliced_data)
sliced_data

array([[-1.16931073e-06, -1.19266998e-06, -1.26044936e-06, ...,
        -7.16440285e-07, -8.44723725e-07, -9.05758950e-07],
       [ 5.71052614e-06,  4.92518233e-06,  4.36646161e-06, ...,
         3.18056730e-06,  4.34903927e-06,  5.61046220e-06],
       [-9.54075528e-07, -9.37561049e-07, -9.09720241e-07, ...,
         7.48116941e-06,  9.76658323e-06,  1.22548854e-05],
       ...,
       [-1.01321872e-05, -9.90499073e-06, -1.03062099e-05, ...,
        -3.92922330e-06, -3.17263257e-06, -1.89901123e-06],
       [-1.15990709e-05, -2.16095432e-06,  1.67647438e-05, ...,
        -1.03270128e-05, -1.20912439e-05, -1.33328636e-05],
       [-1.25586171e-06, -1.09073109e-07,  3.96718882e-07, ...,
         1.86589391e-06,  1.20375281e-06,  1.13471039e-06]])

In [11]:
data=pd.DataFrame(sliced_data)
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.169311e-06,-1.192670e-06,-1.260449e-06,-1.215064e-06,-1.205689e-06,-1.077668e-06,-9.802064e-07,-7.622220e-07,-5.735409e-07,-2.672490e-07,...,1.010905e-06,7.051490e-07,3.826832e-07,1.130196e-07,-9.600296e-07,-3.714728e-07,-5.771173e-07,-7.164403e-07,-8.447237e-07,-9.057589e-07
1,5.710526e-06,4.925182e-06,4.366462e-06,4.142559e-06,4.142163e-06,4.461797e-06,4.983090e-06,5.792004e-06,6.764636e-06,7.978223e-06,...,-1.620005e-06,-1.299833e-06,-8.619781e-07,-3.040478e-07,3.820945e-07,1.191130e-06,2.128661e-06,3.180567e-06,4.349039e-06,5.610462e-06
2,-9.540755e-07,-9.375610e-07,-9.097202e-07,-8.222883e-07,-7.274545e-07,-5.805445e-07,-4.321401e-07,-2.409039e-07,-5.552336e-08,1.622503e-07,...,1.513436e-06,1.121304e-06,1.204625e-06,1.673853e-06,2.594382e-06,3.860632e-06,5.529421e-06,7.481169e-06,9.766583e-06,1.225489e-05
3,6.963214e-06,8.374041e-06,9.837712e-06,1.131185e-05,1.278852e-05,5.017434e-06,-3.610432e-06,-2.351362e-06,-1.213269e-06,-2.560055e-07,...,-1.007485e-06,-1.174187e-06,-1.215175e-06,-1.299252e-06,-1.262560e-06,-1.278325e-06,-1.179648e-06,-1.143671e-06,-9.999766e-07,-9.292095e-07
4,1.499383e-05,1.784484e-05,2.085635e-05,-4.611425e-05,-2.381404e-05,-2.078599e-05,-1.776913e-05,-1.490361e-05,-1.211764e-05,-9.546247e-06,...,5.824750e-07,5.498189e-07,6.393365e-07,5.907119e-07,6.627360e-07,5.914246e-07,6.396869e-07,5.401038e-07,5.599540e-07,1.228616e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,-3.063026e-06,1.921534e-05,-8.854681e-07,-3.128264e-07,-2.159801e-07,5.544465e-07,8.461853e-07,1.805143e-06,2.275919e-06,3.400970e-06,...,6.405011e-06,7.375739e-06,7.704659e-06,8.530197e-06,2.870888e-05,-6.198028e-07,-1.059763e-05,-1.008345e-05,-1.021674e-05,-9.854100e-06
2388,-2.011561e-06,-3.196281e-06,-4.930004e-06,-6.053572e-06,-7.697798e-06,-8.703221e-06,9.799462e-06,8.968753e-06,7.672749e-06,7.067667e-06,...,-2.583465e-06,-3.062207e-06,-4.170647e-06,-4.760976e-06,-5.960595e-06,-6.619525e-06,-7.863365e-06,-8.540970e-06,-9.777318e-06,-1.042142e-05
2389,-1.013219e-05,-9.904991e-06,-1.030621e-05,-1.018729e-05,-1.067930e-05,-1.063173e-05,-1.117384e-05,8.845773e-06,8.298392e-06,8.334979e-06,...,-1.208336e-05,-1.070135e-05,-9.857271e-06,-8.425720e-06,-7.559989e-06,-6.133910e-06,-5.299700e-06,-3.929223e-06,-3.172633e-06,-1.899011e-06
2390,-1.159907e-05,-2.160954e-06,1.676474e-05,1.632419e-05,1.538541e-05,1.509022e-05,1.430197e-05,1.415756e-05,1.351550e-05,1.350808e-05,...,-3.197727e-06,-3.578722e-06,-4.673800e-06,-5.333504e-06,-6.698939e-06,-7.613101e-06,-9.210544e-06,-1.032701e-05,-1.209124e-05,-1.333286e-05


In [12]:
df=data
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.169311e-06,-1.192670e-06,-1.260449e-06,-1.215064e-06,-1.205689e-06,-1.077668e-06,-9.802064e-07,-7.622220e-07,-5.735409e-07,-2.672490e-07,...,1.010905e-06,7.051490e-07,3.826832e-07,1.130196e-07,-9.600296e-07,-3.714728e-07,-5.771173e-07,-7.164403e-07,-8.447237e-07,-9.057589e-07
1,5.710526e-06,4.925182e-06,4.366462e-06,4.142559e-06,4.142163e-06,4.461797e-06,4.983090e-06,5.792004e-06,6.764636e-06,7.978223e-06,...,-1.620005e-06,-1.299833e-06,-8.619781e-07,-3.040478e-07,3.820945e-07,1.191130e-06,2.128661e-06,3.180567e-06,4.349039e-06,5.610462e-06
2,-9.540755e-07,-9.375610e-07,-9.097202e-07,-8.222883e-07,-7.274545e-07,-5.805445e-07,-4.321401e-07,-2.409039e-07,-5.552336e-08,1.622503e-07,...,1.513436e-06,1.121304e-06,1.204625e-06,1.673853e-06,2.594382e-06,3.860632e-06,5.529421e-06,7.481169e-06,9.766583e-06,1.225489e-05
3,6.963214e-06,8.374041e-06,9.837712e-06,1.131185e-05,1.278852e-05,5.017434e-06,-3.610432e-06,-2.351362e-06,-1.213269e-06,-2.560055e-07,...,-1.007485e-06,-1.174187e-06,-1.215175e-06,-1.299252e-06,-1.262560e-06,-1.278325e-06,-1.179648e-06,-1.143671e-06,-9.999766e-07,-9.292095e-07
4,1.499383e-05,1.784484e-05,2.085635e-05,-4.611425e-05,-2.381404e-05,-2.078599e-05,-1.776913e-05,-1.490361e-05,-1.211764e-05,-9.546247e-06,...,5.824750e-07,5.498189e-07,6.393365e-07,5.907119e-07,6.627360e-07,5.914246e-07,6.396869e-07,5.401038e-07,5.599540e-07,1.228616e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,-3.063026e-06,1.921534e-05,-8.854681e-07,-3.128264e-07,-2.159801e-07,5.544465e-07,8.461853e-07,1.805143e-06,2.275919e-06,3.400970e-06,...,6.405011e-06,7.375739e-06,7.704659e-06,8.530197e-06,2.870888e-05,-6.198028e-07,-1.059763e-05,-1.008345e-05,-1.021674e-05,-9.854100e-06
2388,-2.011561e-06,-3.196281e-06,-4.930004e-06,-6.053572e-06,-7.697798e-06,-8.703221e-06,9.799462e-06,8.968753e-06,7.672749e-06,7.067667e-06,...,-2.583465e-06,-3.062207e-06,-4.170647e-06,-4.760976e-06,-5.960595e-06,-6.619525e-06,-7.863365e-06,-8.540970e-06,-9.777318e-06,-1.042142e-05
2389,-1.013219e-05,-9.904991e-06,-1.030621e-05,-1.018729e-05,-1.067930e-05,-1.063173e-05,-1.117384e-05,8.845773e-06,8.298392e-06,8.334979e-06,...,-1.208336e-05,-1.070135e-05,-9.857271e-06,-8.425720e-06,-7.559989e-06,-6.133910e-06,-5.299700e-06,-3.929223e-06,-3.172633e-06,-1.899011e-06
2390,-1.159907e-05,-2.160954e-06,1.676474e-05,1.632419e-05,1.538541e-05,1.509022e-05,1.430197e-05,1.415756e-05,1.351550e-05,1.350808e-05,...,-3.197727e-06,-3.578722e-06,-4.673800e-06,-5.333504e-06,-6.698939e-06,-7.613101e-06,-9.210544e-06,-1.032701e-05,-1.209124e-05,-1.333286e-05


In [13]:
# Set the wavelet and decomposition level
wavelet = 'db4'
level = 3
feature_matrix=[]

In [14]:
# Loop over each window
for window_data in sliced_data:
    # Extract time domain features from the window data
    time_feats = [np.mean(window_data), np.std(window_data)]
    
    # Compute the DWT coefficients
    coeffs = pywt.wavedec(window_data, wavelet, level=level)
    
    # Extract features from the DWT coefficients
    dwt_features = []
    for i, coeff in enumerate(coeffs):
        # Compute the standard deviation of the current coefficient
        std_dev = np.std(coeff)
        dwt_features.append(std_dev)
        
        # Compute the mean of the absolute values of the current coefficient
        mean_abs = np.mean(np.abs(coeff))
        dwt_features.append(mean_abs)
    
    # Combine time domain and DWT features
    features = time_feats + dwt_features
    
    # Add the features for the current window to the feature matrix
    feature_matrix.append(features)

In [15]:
data5=pd.DataFrame(feature_matrix)

In [16]:
data5

,0,1,2,3,4,5,6,7,8,9
0,9.133100e-08,0.000008,0.000013,0.000010,0.000009,0.000005,0.000005,2.217326e-06,0.000003,1.060242e-06
1,-1.258124e-07,0.000005,0.000011,0.000010,0.000004,0.000002,0.000002,8.491899e-07,0.000002,6.717820e-07
2,5.765124e-07,0.000009,0.000015,0.000010,0.000012,0.000007,0.000008,3.064267e-06,0.000002,6.955545e-07
3,9.289399e-08,0.000011,0.000024,0.000020,0.000013,0.000006,0.000007,3.137116e-06,0.000004,1.341024e-06
4,-5.328161e-07,0.000007,0.000012,0.000007,0.000020,0.000008,0.000008,2.978667e-06,0.000004,1.139864e-06
...,...,...,...,...,...,...,...,...,...,...
2387,3.500688e-07,0.000008,0.000017,0.000014,0.000011,0.000009,0.000005,3.622880e-06,0.000004,2.467616e-06
2388,-3.698112e-07,0.000008,0.000016,0.000014,0.000009,0.000007,0.000007,4.613027e-06,0.000004,1.768716e-06
2389,-2.983017e-07,0.000013,0.000026,0.000024,0.000017,0.000013,0.000007,4.479975e-06,0.000004,1.944267e-06
2390,6.691910e-08,0.000013,0.000025,0.000021,0.000016,0.000012,0.000009,6.737977e-06,0.000005,2.261113e-06


In [17]:
from sklearn.decomposition import PCA
# Apply PCA to the feature matrix
n_components = 4
pca = PCA(n_components=n_components)
reduced_feature_matrix = pca.fit_transform(feature_matrix)

In [18]:
data5=pd.DataFrame(reduced_feature_matrix)
data5

,0,1,2,3
0,-2.079024e-05,-3.166390e-07,-1.377979e-06,3.863017e-07
1,-2.629613e-05,4.598534e-06,-1.489110e-06,6.813670e-07
2,-1.677903e-05,-3.372569e-06,-7.059989e-07,-1.842810e-06
3,-5.526736e-06,3.581135e-06,-1.727936e-06,3.352294e-08
4,-1.721738e-05,-1.060639e-05,-3.921169e-06,-2.589434e-06
...,...,...,...,...
2387,-1.382102e-05,-3.057441e-07,-2.218128e-06,-1.025463e-07
2388,-1.540635e-05,6.479533e-07,1.075848e-06,-1.497932e-06
2389,1.490886e-06,9.956865e-07,-4.951140e-06,-2.524179e-06
2390,-3.331976e-08,-1.162739e-06,-1.247425e-06,-2.268117e-06


In [19]:
data5[''] = 0
data5

,0,1,2,3,
0,-2.079024e-05,-3.166390e-07,-1.377979e-06,3.863017e-07,0
1,-2.629613e-05,4.598534e-06,-1.489110e-06,6.813670e-07,0
2,-1.677903e-05,-3.372569e-06,-7.059989e-07,-1.842810e-06,0
3,-5.526736e-06,3.581135e-06,-1.727936e-06,3.352294e-08,0
4,-1.721738e-05,-1.060639e-05,-3.921169e-06,-2.589434e-06,0
...,...,...,...,...,...
2387,-1.382102e-05,-3.057441e-07,-2.218128e-06,-1.025463e-07,0
2388,-1.540635e-05,6.479533e-07,1.075848e-06,-1.497932e-06,0
2389,1.490886e-06,9.956865e-07,-4.951140e-06,-2.524179e-06,0
2390,-3.331976e-08,-1.162739e-06,-1.247425e-06,-2.268117e-06,0


In [20]:
# Save the filtered data to a CSV file
data5.to_csv('Final_Class0.csv', index=False)

# Class 1 Wrist Flexing 

In [21]:
df = pd.read_csv('D:/TY Sem 2/EDI/15k/class.3.csv')
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,-0.00012,-0.00014,0.00008,0.00005,0.00012,0.00008,0.00006,-0.00007
1,-0.00012,-0.00014,0.00008,0.00005,0.00012,0.00008,0.00006,-0.00007
2,-0.00012,-0.00014,0.00008,0.00005,0.00012,0.00008,0.00006,-0.00007
3,-0.00012,-0.00014,0.00008,0.00005,0.00012,0.00008,0.00006,-0.00007
4,-0.00012,-0.00014,0.00008,0.00005,0.00012,0.00008,0.00006,-0.00007
...,...,...,...,...,...,...,...,...
14995,0.00025,0.00050,0.00011,0.00010,0.00035,-0.00003,-0.00002,-0.00010
14996,0.00025,0.00050,0.00011,0.00010,0.00035,-0.00003,-0.00002,-0.00010
14997,0.00025,0.00050,0.00011,0.00010,0.00035,-0.00003,-0.00002,-0.00010
14998,0.00025,0.00050,0.00011,0.00010,0.00035,-0.00003,-0.00002,-0.00010


In [22]:
# Define the notch frequency and Q-factor
f0 = 60  # Notch frequency (Hz)
Q = 10   # Q-factor
# Define the filter parameters
fs = 801       # Sampling frequency (Hz)
lowcut = 20    # Low cutoff frequency (Hz)
highcut = 400  # High cutoff frequency (Hz)
order = 4      # Filter order


In [23]:
# Apply the filters
for col in df.columns:
    # Get the EMG signal from the current column
    emg = df[col].values
    
    # Apply the Butterworth filter
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    emg_filtered = signal.filtfilt(b, a, emg)
    
    # Apply the notch filter
    wo = f0 / (fs / 2)  # Normalized frequency
    bw = wo / Q
    b_notch, a_notch = signal.iirnotch(wo, bw)
    emg_filtered_notched = signal.filtfilt(b_notch, a_notch, emg_filtered)
    
    # Replace the original signal with the filtered signal
    df[col] = emg_filtered_notched

In [24]:
emg_filtered_notched

array([ 1.96597823e-06, -1.37845629e-06, -2.33148376e-06, ...,
       -1.01337696e-05, -1.73049281e-05, -1.06956270e-05])

In [25]:
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,3.704528e-06,9.225303e-07,6.913105e-07,-1.420832e-07,-0.000002,0.000002,1.772164e-06,0.000002
1,-3.959573e-07,-3.175131e-06,1.291529e-05,3.967786e-06,0.000019,0.000015,1.520585e-05,-0.000001
2,-4.170384e-06,-8.342496e-06,2.531038e-05,1.165820e-05,0.000044,0.000032,3.005453e-05,-0.000002
3,-8.779469e-06,-1.261605e-05,3.709221e-05,1.562000e-05,0.000064,0.000045,4.280497e-05,-0.000006
4,-1.332682e-05,-1.801553e-05,4.877458e-05,2.306799e-05,0.000088,0.000061,5.656653e-05,-0.000007
...,...,...,...,...,...,...,...,...
14995,1.446737e-04,6.698625e-05,2.259203e-05,2.251859e-05,0.000082,0.000008,-3.440667e-06,-0.000011
14996,8.475871e-05,4.222868e-05,8.967433e-06,1.662962e-05,0.000066,0.000013,-1.447035e-07,-0.000017
14997,9.889643e-05,4.425295e-05,1.613496e-05,1.425755e-05,0.000051,-0.000003,-8.328192e-06,-0.000010
14998,4.056676e-05,1.893710e-05,2.723286e-06,8.746486e-06,0.000036,0.000003,-5.690127e-06,-0.000017


In [26]:
# Set the window size and overlap
window_size = 100   # number of samples in each window
overlap = 50       # number of samples to overlap between windows

In [27]:
# Get the number of rows and columns in the DataFrame
num_rows, num_cols = df.shape

In [28]:
# Initialize a list to store the sliced data
sliced_data = []
for col in range(num_cols):
    # Get the data for the current channel
    channel_data = df.iloc[:, col].values
    
    # Compute the number of windows
    num_windows = int(np.ceil((num_rows - window_size) / overlap)) + 1
    
    # Loop over each window
    for i in range(num_windows):
        # Compute the start and end indices of the window
        start_index = i * overlap
        end_index = start_index + window_size
        
        # Make sure the end index does not exceed the number of rows
        if end_index > num_rows:
            end_index = num_rows
        
        # Get the data for the current window
        window_data = channel_data[start_index:end_index]
        
        # Add the window data to the sliced data list
        sliced_data.append(window_data)

In [29]:
# Convert the sliced data list to a numpy array
sliced_data = np.array(sliced_data)
sliced_data

array([[ 3.70452766e-06, -3.95957298e-07, -4.17038416e-06, ...,
         1.79693757e-04,  1.75290154e-04,  1.72521369e-04],
       [-8.44272721e-05, -6.91391927e-05, -5.34464429e-05, ...,
        -3.79018122e-05, -3.34346355e-05, -3.09218879e-05],
       [ 1.73326799e-04,  1.76133432e-04,  1.82783668e-04, ...,
         2.37192136e-05,  3.06489629e-05,  3.60621663e-05],
       ...,
       [-3.49659210e-05, -4.74786186e-05, -5.25656968e-05, ...,
        -1.30802322e-06, -4.42231851e-06, -7.41512709e-06],
       [ 1.19535912e-04,  1.06820730e-04,  9.85883499e-05, ...,
        -9.53517564e-06, -3.35483979e-05, -3.10655039e-05],
       [-1.07573507e-05, -1.36725478e-05, -1.71093714e-05, ...,
        -1.01337696e-05, -1.73049281e-05, -1.06956270e-05]])

In [30]:
data=pd.DataFrame(sliced_data)
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000004,-3.959573e-07,-0.000004,-0.000009,-0.000013,0.000066,0.000060,0.000053,0.000046,0.000036,...,-0.000101,-0.000115,-0.000128,-0.000142,-0.000153,-0.000444,-0.000453,0.000180,0.000175,0.000173
1,-0.000084,-6.913919e-05,-0.000053,-0.000039,-0.000024,-0.000011,0.000003,0.000015,-0.000032,-0.000022,...,-0.000070,-0.000067,-0.000062,-0.000058,-0.000052,-0.000048,-0.000042,-0.000038,-0.000033,-0.000031
2,0.000173,1.761334e-04,0.000183,0.000192,0.000204,0.000219,0.000237,0.000257,0.000280,0.000303,...,-0.000025,-0.000019,-0.000011,-0.000004,0.000004,0.000010,0.000018,0.000024,0.000031,0.000036
3,-0.000029,-2.856756e-05,-0.000029,-0.000032,-0.000037,-0.000044,-0.000052,-0.000062,-0.000074,-0.000089,...,-0.000391,0.000047,0.000045,0.000041,0.000038,0.000033,0.000030,0.000024,0.000019,0.000013
4,0.000042,4.695224e-05,0.000052,0.000056,-0.000089,-0.000026,-0.000022,-0.000020,-0.000016,-0.000015,...,0.000036,0.000027,0.000028,0.000028,0.000028,0.000025,0.000023,0.000018,0.000013,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.000009,-7.319974e-06,-0.000005,-0.000017,-0.000010,-0.000396,0.000236,0.000235,0.000252,0.000255,...,-0.000052,-0.000050,-0.000039,-0.000039,-0.000029,-0.000031,-0.000024,-0.000029,-0.000026,-0.000034
2388,0.000080,-3.718970e-04,-0.000349,0.000012,0.000037,0.000049,0.000075,0.000087,0.000113,0.000124,...,-0.000090,-0.000088,-0.000082,-0.000084,-0.000082,-0.000087,0.000131,-0.000357,0.000138,0.000127
2389,-0.000035,-4.747862e-05,-0.000053,-0.000069,-0.000078,-0.000098,-0.000111,-0.000134,-0.000149,-0.000173,...,-0.000185,0.000253,0.000012,0.000009,0.000007,0.000004,0.000002,-0.000001,-0.000004,-0.000007
2390,0.000120,1.068207e-04,0.000099,0.000085,0.000077,0.000064,0.000056,0.000044,0.000038,0.000028,...,-0.000010,-0.000007,-0.000010,-0.000008,-0.000011,-0.000008,-0.000012,-0.000010,-0.000034,-0.000031


In [31]:
df=data
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000004,-3.959573e-07,-0.000004,-0.000009,-0.000013,0.000066,0.000060,0.000053,0.000046,0.000036,...,-0.000101,-0.000115,-0.000128,-0.000142,-0.000153,-0.000444,-0.000453,0.000180,0.000175,0.000173
1,-0.000084,-6.913919e-05,-0.000053,-0.000039,-0.000024,-0.000011,0.000003,0.000015,-0.000032,-0.000022,...,-0.000070,-0.000067,-0.000062,-0.000058,-0.000052,-0.000048,-0.000042,-0.000038,-0.000033,-0.000031
2,0.000173,1.761334e-04,0.000183,0.000192,0.000204,0.000219,0.000237,0.000257,0.000280,0.000303,...,-0.000025,-0.000019,-0.000011,-0.000004,0.000004,0.000010,0.000018,0.000024,0.000031,0.000036
3,-0.000029,-2.856756e-05,-0.000029,-0.000032,-0.000037,-0.000044,-0.000052,-0.000062,-0.000074,-0.000089,...,-0.000391,0.000047,0.000045,0.000041,0.000038,0.000033,0.000030,0.000024,0.000019,0.000013
4,0.000042,4.695224e-05,0.000052,0.000056,-0.000089,-0.000026,-0.000022,-0.000020,-0.000016,-0.000015,...,0.000036,0.000027,0.000028,0.000028,0.000028,0.000025,0.000023,0.000018,0.000013,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.000009,-7.319974e-06,-0.000005,-0.000017,-0.000010,-0.000396,0.000236,0.000235,0.000252,0.000255,...,-0.000052,-0.000050,-0.000039,-0.000039,-0.000029,-0.000031,-0.000024,-0.000029,-0.000026,-0.000034
2388,0.000080,-3.718970e-04,-0.000349,0.000012,0.000037,0.000049,0.000075,0.000087,0.000113,0.000124,...,-0.000090,-0.000088,-0.000082,-0.000084,-0.000082,-0.000087,0.000131,-0.000357,0.000138,0.000127
2389,-0.000035,-4.747862e-05,-0.000053,-0.000069,-0.000078,-0.000098,-0.000111,-0.000134,-0.000149,-0.000173,...,-0.000185,0.000253,0.000012,0.000009,0.000007,0.000004,0.000002,-0.000001,-0.000004,-0.000007
2390,0.000120,1.068207e-04,0.000099,0.000085,0.000077,0.000064,0.000056,0.000044,0.000038,0.000028,...,-0.000010,-0.000007,-0.000010,-0.000008,-0.000011,-0.000008,-0.000012,-0.000010,-0.000034,-0.000031


In [32]:
# Set the wavelet and decomposition level
wavelet = 'db4'
level = 3
feature_matrix=[]

In [33]:
# Loop over each window
for window_data in sliced_data:
    # Extract time domain features from the window data
    time_feats = [np.mean(window_data), np.std(window_data)]
    
    # Compute the DWT coefficients
    coeffs = pywt.wavedec(window_data, wavelet, level=level)
    
    # Extract features from the DWT coefficients
    dwt_features = []
    for i, coeff in enumerate(coeffs):
        # Compute the standard deviation of the current coefficient
        std_dev = np.std(coeff)
        dwt_features.append(std_dev)
        
        # Compute the mean of the absolute values of the current coefficient
        mean_abs = np.mean(np.abs(coeff))
        dwt_features.append(mean_abs)
    
    # Combine time domain and DWT features
    features = time_feats + dwt_features
    
    # Add the features for the current window to the feature matrix
    feature_matrix.append(features)

In [34]:
data5=pd.DataFrame(feature_matrix)

In [35]:
data5

,0,1,2,3,4,5,6,7,8,9
0,-0.000010,0.000097,0.000195,0.000129,0.000136,0.000088,0.000118,0.000052,0.000041,0.000018
1,0.000002,0.000187,0.000328,0.000233,0.000237,0.000163,0.000132,0.000073,0.000057,0.000024
2,0.000009,0.000184,0.000422,0.000369,0.000178,0.000115,0.000077,0.000045,0.000072,0.000030
3,-0.000003,0.000095,0.000119,0.000091,0.000156,0.000089,0.000108,0.000055,0.000054,0.000019
4,0.000003,0.000062,0.000076,0.000061,0.000052,0.000032,0.000087,0.000031,0.000025,0.000010
...,...,...,...,...,...,...,...,...,...,...
2387,0.000005,0.000172,0.000260,0.000193,0.000189,0.000146,0.000140,0.000094,0.000101,0.000044
2388,-0.000010,0.000149,0.000276,0.000261,0.000150,0.000114,0.000143,0.000089,0.000117,0.000054
2389,-0.000002,0.000102,0.000169,0.000145,0.000094,0.000064,0.000072,0.000044,0.000080,0.000035
2390,0.000004,0.000053,0.000136,0.000112,0.000037,0.000031,0.000035,0.000020,0.000052,0.000018


In [36]:
from sklearn.decomposition import PCA
# Apply PCA to the feature matrix
n_components = 4
pca = PCA(n_components=n_components)
reduced_feature_matrix = pca.fit_transform(feature_matrix)

In [37]:
data5=pd.DataFrame(reduced_feature_matrix)
data5

,0,1,2,3
0,0.000056,-0.000035,0.000020,-0.000027
1,0.000281,-0.000054,-0.000019,-0.000013
2,0.000362,0.000109,-0.000036,0.000024
3,-0.000003,-0.000094,0.000016,-0.000019
4,-0.000118,-0.000018,0.000042,-0.000023
...,...,...,...,...
2387,0.000205,-0.000074,0.000041,0.000006
2388,0.000223,-0.000002,0.000060,0.000003
2389,0.000025,-0.000001,0.000029,0.000025
2390,-0.000070,0.000046,0.000015,0.000021


In [38]:
data5[''] = 1
data5

,0,1,2,3,
0,0.000056,-0.000035,0.000020,-0.000027,1
1,0.000281,-0.000054,-0.000019,-0.000013,1
2,0.000362,0.000109,-0.000036,0.000024,1
3,-0.000003,-0.000094,0.000016,-0.000019,1
4,-0.000118,-0.000018,0.000042,-0.000023,1
...,...,...,...,...,...
2387,0.000205,-0.000074,0.000041,0.000006,1
2388,0.000223,-0.000002,0.000060,0.000003,1
2389,0.000025,-0.000001,0.000029,0.000025,1
2390,-0.000070,0.000046,0.000015,0.000021,1


In [39]:
# Save the filtered data to a CSV file
data5.to_csv('Final_Class1.csv', index=False)

# Class 2 Wrist Extension

In [40]:
df = pd.read_csv('D:/TY Sem 2/EDI/15k/class.4.csv')
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,-0.00014,-0.00029,-0.00036,-0.00047,-0.00075,-0.00066,-0.00010,-0.00010
1,-0.00014,-0.00029,-0.00036,-0.00047,-0.00075,-0.00066,-0.00010,-0.00010
2,-0.00014,-0.00029,-0.00036,-0.00047,-0.00075,-0.00066,-0.00010,-0.00010
3,-0.00014,-0.00029,-0.00036,-0.00047,-0.00075,-0.00066,-0.00010,-0.00010
4,-0.00014,-0.00029,-0.00036,-0.00047,-0.00075,-0.00066,-0.00010,-0.00010
...,...,...,...,...,...,...,...,...
14995,0.00006,0.00010,0.00002,-0.00043,-0.00080,0.00031,0.00011,0.00007
14996,0.00006,0.00010,0.00002,-0.00043,-0.00080,0.00031,0.00011,0.00007
14997,0.00006,0.00010,0.00002,-0.00043,-0.00080,0.00031,0.00011,0.00007
14998,0.00006,0.00010,0.00002,-0.00043,-0.00080,0.00031,0.00011,0.00007


In [41]:
# Define the notch frequency and Q-factor
f0 = 60  # Notch frequency (Hz)
Q = 10   # Q-factor
# Define the filter parameters
fs = 801       # Sampling frequency (Hz)
lowcut = 20    # Low cutoff frequency (Hz)
highcut = 400  # High cutoff frequency (Hz)
order = 4      # Filter order


In [42]:
# Apply the filters
for col in df.columns:
    # Get the EMG signal from the current column
    emg = df[col].values
    
    # Apply the Butterworth filter
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    emg_filtered = signal.filtfilt(b, a, emg)
    
    # Apply the notch filter
    wo = f0 / (fs / 2)  # Normalized frequency
    bw = wo / Q
    b_notch, a_notch = signal.iirnotch(wo, bw)
    emg_filtered_notched = signal.filtfilt(b_notch, a_notch, emg_filtered)
    
    # Replace the original signal with the filtered signal
    df[col] = emg_filtered_notched

In [43]:
emg_filtered_notched

array([1.18230133e-06, 5.76663495e-06, 9.65119782e-06, ...,
       5.15002810e-05, 5.11361958e-05, 5.04989982e-05])

In [44]:
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,1.560785e-07,-7.065642e-07,-0.000001,-0.000007,-0.000005,-0.000004,-0.000001,0.000001
1,-3.018772e-06,-1.301659e-05,-0.000018,-0.000029,-0.000037,-0.000024,0.000007,0.000006
2,-7.573588e-06,-2.682323e-05,-0.000037,-0.000054,-0.000077,-0.000057,0.000011,0.000010
3,-1.066820e-05,-3.901936e-05,-0.000054,-0.000076,-0.000109,-0.000076,0.000019,0.000014
4,-1.517656e-05,-5.268072e-05,-0.000073,-0.000100,-0.000150,-0.000108,0.000024,0.000018
...,...,...,...,...,...,...,...,...
14995,3.655770e-05,4.513885e-05,0.000079,0.000075,0.000079,0.000034,0.000040,0.000050
14996,3.458199e-05,4.336821e-05,0.000074,0.000090,0.000096,0.000074,0.000051,0.000051
14997,3.738441e-05,4.524200e-05,0.000077,0.000066,0.000089,0.000052,0.000043,0.000052
14998,3.462042e-05,4.257179e-05,0.000070,0.000076,0.000099,0.000089,0.000053,0.000051


In [45]:
# Set the window size and overlap
window_size = 100   # number of samples in each window
overlap = 50       # number of samples to overlap between windows

In [46]:
# Get the number of rows and columns in the DataFrame
num_rows, num_cols = df.shape

In [47]:
# Initialize a list to store the sliced data
sliced_data = []
for col in range(num_cols):
    # Get the data for the current channel
    channel_data = df.iloc[:, col].values
    
    # Compute the number of windows
    num_windows = int(np.ceil((num_rows - window_size) / overlap)) + 1
    
    # Loop over each window
    for i in range(num_windows):
        # Compute the start and end indices of the window
        start_index = i * overlap
        end_index = start_index + window_size
        
        # Make sure the end index does not exceed the number of rows
        if end_index > num_rows:
            end_index = num_rows
        
        # Get the data for the current window
        window_data = channel_data[start_index:end_index]
        
        # Add the window data to the sliced data list
        sliced_data.append(window_data)

In [48]:
# Convert the sliced data list to a numpy array
sliced_data = np.array(sliced_data)
sliced_data

array([[ 1.56078460e-07, -3.01877158e-06, -7.57358845e-06, ...,
         1.01086981e-05,  1.02814967e-05,  1.26679544e-05],
       [-6.04030831e-05, -5.67299469e-05, -5.45041644e-05, ...,
         2.05034653e-05,  1.53301718e-05,  1.29998842e-05],
       [ 1.35770553e-05,  1.66549139e-05,  1.81758959e-05, ...,
        -1.37013966e-05, -1.83364543e-05, -2.06020778e-05],
       ...,
       [-3.01595724e-05, -3.30147203e-05, -3.78028687e-05, ...,
         3.98822219e-05, -4.27287353e-05, -4.28710995e-05],
       [ 4.15145097e-06,  3.55129063e-06,  1.52566123e-06, ...,
        -2.17598794e-05, -3.20327656e-05, -4.20149386e-05],
       [-4.39001772e-05, -4.32213921e-05, -4.34713864e-05, ...,
         5.15002810e-05,  5.11361958e-05,  5.04989982e-05]])

In [49]:
data=pd.DataFrame(sliced_data)
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.560785e-07,-0.000003,-0.000008,-0.000011,-1.517656e-05,-1.824372e-05,-2.276053e-05,-2.585504e-05,-0.000030,-3.360438e-05,...,0.000012,0.000012,0.000010,9.685458e-06,8.305143e-06,9.117358e-06,0.000009,0.000010,0.000010,0.000013
1,-6.040308e-05,-0.000057,-0.000055,-0.000050,-4.799929e-05,-1.337734e-04,3.886521e-05,4.319348e-05,0.000046,5.021010e-05,...,-0.000045,-0.000052,-0.000090,-9.562105e-05,3.606399e-05,3.093648e-05,0.000024,0.000021,0.000015,0.000013
2,1.357706e-05,0.000017,0.000018,0.000022,2.373093e-05,2.765926e-05,2.978735e-05,3.776489e-06,-0.000034,-3.040464e-05,...,0.000009,0.000007,0.000003,6.110694e-07,-4.046108e-06,-6.518448e-06,-0.000011,-0.000014,-0.000018,-0.000021
3,9.469220e-06,0.000009,0.000007,0.000008,8.709189e-06,1.195917e-05,1.408212e-05,1.920702e-05,0.000023,2.995260e-05,...,0.000033,0.000034,0.000032,2.444986e-06,3.670895e-07,5.449821e-07,-0.000002,-0.000002,-0.000005,-0.000006
4,-2.499003e-05,0.000043,0.000039,0.000008,3.963557e-06,2.944370e-06,-9.870304e-08,-5.734298e-07,-0.000003,-2.985347e-06,...,-0.000026,-0.000027,-0.000030,-3.156699e-05,-3.596338e-05,-3.807153e-05,0.000037,0.000034,0.000029,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,1.680907e-05,0.000020,0.000022,0.000026,2.854420e-05,3.328260e-05,3.642157e-05,4.153641e-05,0.000045,1.006673e-05,...,-0.000017,-0.000015,-0.000015,-1.454841e-05,-1.603563e-05,-1.632377e-05,-0.000019,-0.000020,-0.000024,-0.000026
2388,3.427526e-05,0.000030,0.000025,0.000022,1.720187e-05,1.487945e-05,1.123285e-05,9.884410e-06,0.000007,6.916019e-06,...,0.000012,0.000013,0.000013,1.323499e-05,1.186735e-05,1.175738e-05,0.000010,0.000009,0.000007,0.000006
2389,-3.015957e-05,-0.000033,-0.000038,-0.000041,-7.526132e-05,1.553624e-06,2.682316e-05,2.405863e-05,0.000020,1.800995e-05,...,0.000061,0.000057,0.000053,4.965884e-05,4.581889e-05,4.380280e-05,0.000041,0.000040,-0.000043,-0.000043
2390,4.151451e-06,0.000004,0.000002,0.000001,-4.500358e-07,-3.539047e-07,-1.558131e-06,-9.978827e-07,-0.000002,-7.324101e-07,...,0.000012,0.000009,0.000004,-1.053009e-06,4.141846e-05,3.538035e-05,-0.000013,-0.000022,-0.000032,-0.000042


In [50]:
df=data
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.560785e-07,-0.000003,-0.000008,-0.000011,-1.517656e-05,-1.824372e-05,-2.276053e-05,-2.585504e-05,-0.000030,-3.360438e-05,...,0.000012,0.000012,0.000010,9.685458e-06,8.305143e-06,9.117358e-06,0.000009,0.000010,0.000010,0.000013
1,-6.040308e-05,-0.000057,-0.000055,-0.000050,-4.799929e-05,-1.337734e-04,3.886521e-05,4.319348e-05,0.000046,5.021010e-05,...,-0.000045,-0.000052,-0.000090,-9.562105e-05,3.606399e-05,3.093648e-05,0.000024,0.000021,0.000015,0.000013
2,1.357706e-05,0.000017,0.000018,0.000022,2.373093e-05,2.765926e-05,2.978735e-05,3.776489e-06,-0.000034,-3.040464e-05,...,0.000009,0.000007,0.000003,6.110694e-07,-4.046108e-06,-6.518448e-06,-0.000011,-0.000014,-0.000018,-0.000021
3,9.469220e-06,0.000009,0.000007,0.000008,8.709189e-06,1.195917e-05,1.408212e-05,1.920702e-05,0.000023,2.995260e-05,...,0.000033,0.000034,0.000032,2.444986e-06,3.670895e-07,5.449821e-07,-0.000002,-0.000002,-0.000005,-0.000006
4,-2.499003e-05,0.000043,0.000039,0.000008,3.963557e-06,2.944370e-06,-9.870304e-08,-5.734298e-07,-0.000003,-2.985347e-06,...,-0.000026,-0.000027,-0.000030,-3.156699e-05,-3.596338e-05,-3.807153e-05,0.000037,0.000034,0.000029,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,1.680907e-05,0.000020,0.000022,0.000026,2.854420e-05,3.328260e-05,3.642157e-05,4.153641e-05,0.000045,1.006673e-05,...,-0.000017,-0.000015,-0.000015,-1.454841e-05,-1.603563e-05,-1.632377e-05,-0.000019,-0.000020,-0.000024,-0.000026
2388,3.427526e-05,0.000030,0.000025,0.000022,1.720187e-05,1.487945e-05,1.123285e-05,9.884410e-06,0.000007,6.916019e-06,...,0.000012,0.000013,0.000013,1.323499e-05,1.186735e-05,1.175738e-05,0.000010,0.000009,0.000007,0.000006
2389,-3.015957e-05,-0.000033,-0.000038,-0.000041,-7.526132e-05,1.553624e-06,2.682316e-05,2.405863e-05,0.000020,1.800995e-05,...,0.000061,0.000057,0.000053,4.965884e-05,4.581889e-05,4.380280e-05,0.000041,0.000040,-0.000043,-0.000043
2390,4.151451e-06,0.000004,0.000002,0.000001,-4.500358e-07,-3.539047e-07,-1.558131e-06,-9.978827e-07,-0.000002,-7.324101e-07,...,0.000012,0.000009,0.000004,-1.053009e-06,4.141846e-05,3.538035e-05,-0.000013,-0.000022,-0.000032,-0.000042


In [51]:
# Set the wavelet and decomposition level
wavelet = 'db4'
level = 3
feature_matrix=[]

In [52]:
# Loop over each window
for window_data in sliced_data:
    # Extract time domain features from the window data
    time_feats = [np.mean(window_data), np.std(window_data)]
    
    # Compute the DWT coefficients
    coeffs = pywt.wavedec(window_data, wavelet, level=level)
    
    # Extract features from the DWT coefficients
    dwt_features = []
    for i, coeff in enumerate(coeffs):
        # Compute the standard deviation of the current coefficient
        std_dev = np.std(coeff)
        dwt_features.append(std_dev)
        
        # Compute the mean of the absolute values of the current coefficient
        mean_abs = np.mean(np.abs(coeff))
        dwt_features.append(mean_abs)
    
    # Combine time domain and DWT features
    features = time_feats + dwt_features
    
    # Add the features for the current window to the feature matrix
    feature_matrix.append(features)

In [53]:
data5=pd.DataFrame(feature_matrix)

In [54]:
data5

,0,1,2,3,4,5,6,7,8,9
0,-7.525763e-07,0.000032,0.000049,0.000039,0.000044,0.000030,0.000017,0.000011,0.000018,0.000009
1,-2.339028e-06,0.000033,0.000090,0.000080,0.000038,0.000029,0.000026,0.000018,0.000020,0.000010
2,-5.504090e-07,0.000030,0.000059,0.000050,0.000034,0.000025,0.000016,0.000010,0.000012,0.000005
3,1.450166e-06,0.000023,0.000030,0.000023,0.000039,0.000026,0.000019,0.000010,0.000007,0.000004
4,-1.975338e-07,0.000019,0.000044,0.000039,0.000021,0.000016,0.000018,0.000012,0.000009,0.000005
...,...,...,...,...,...,...,...,...,...,...
2387,-2.577186e-07,0.000046,0.000089,0.000072,0.000039,0.000031,0.000033,0.000022,0.000023,0.000010
2388,8.670617e-07,0.000043,0.000066,0.000060,0.000058,0.000045,0.000034,0.000023,0.000029,0.000013
2389,2.166661e-06,0.000047,0.000094,0.000080,0.000040,0.000034,0.000037,0.000024,0.000033,0.000014
2390,1.112547e-06,0.000035,0.000063,0.000042,0.000034,0.000027,0.000024,0.000012,0.000024,0.000009


In [55]:
from sklearn.decomposition import PCA
# Apply PCA to the feature matrix
n_components = 4
pca = PCA(n_components=n_components)
reduced_feature_matrix = pca.fit_transform(feature_matrix)

In [56]:
data5=pd.DataFrame(reduced_feature_matrix)
data5

,0,1,2,3
0,-0.000238,0.000008,-1.098566e-05,-3.094375e-06
1,-0.000190,-0.000025,-2.928873e-06,4.443215e-06
2,-0.000232,-0.000011,-9.544331e-06,1.550101e-06
3,-0.000265,0.000013,-1.077376e-05,7.368191e-06
4,-0.000257,-0.000013,-6.182824e-07,6.888443e-06
...,...,...,...,...
2387,-0.000188,-0.000015,5.664910e-06,8.595146e-08
2388,-0.000198,0.000017,-2.938213e-06,1.938712e-06
2389,-0.000177,-0.000014,1.063117e-05,-2.615482e-06
2390,-0.000228,-0.000002,1.678523e-06,-6.784367e-06


In [57]:
data5[''] = 2
data5

,0,1,2,3,
0,-0.000238,0.000008,-1.098566e-05,-3.094375e-06,2
1,-0.000190,-0.000025,-2.928873e-06,4.443215e-06,2
2,-0.000232,-0.000011,-9.544331e-06,1.550101e-06,2
3,-0.000265,0.000013,-1.077376e-05,7.368191e-06,2
4,-0.000257,-0.000013,-6.182824e-07,6.888443e-06,2
...,...,...,...,...,...
2387,-0.000188,-0.000015,5.664910e-06,8.595146e-08,2
2388,-0.000198,0.000017,-2.938213e-06,1.938712e-06,2
2389,-0.000177,-0.000014,1.063117e-05,-2.615482e-06,2
2390,-0.000228,-0.000002,1.678523e-06,-6.784367e-06,2


In [58]:
# Save the filtered data to a CSV file
data5.to_csv('Final_Class2.csv', index=False)

# Classification

In [59]:

# Load the CSV file into a DataFrame
df1 = pd.read_csv('Final_Class0.csv',header=None, skiprows=1)

In [60]:
df1

,0,1,2,3,4
0,-2.079024e-05,-3.166390e-07,-1.377979e-06,3.863017e-07,0
1,-2.629613e-05,4.598534e-06,-1.489110e-06,6.813670e-07,0
2,-1.677903e-05,-3.372569e-06,-7.059989e-07,-1.842810e-06,0
3,-5.526736e-06,3.581135e-06,-1.727936e-06,3.352294e-08,0
4,-1.721738e-05,-1.060639e-05,-3.921169e-06,-2.589434e-06,0
...,...,...,...,...,...
2387,-1.382102e-05,-3.057441e-07,-2.218128e-06,-1.025463e-07,0
2388,-1.540635e-05,6.479533e-07,1.075848e-06,-1.497932e-06,0
2389,1.490886e-06,9.956865e-07,-4.951140e-06,-2.524179e-06,0
2390,-3.331976e-08,-1.162739e-06,-1.247425e-06,-2.268117e-06,0


In [61]:

# Load the CSV file into a DataFrame
df2 = pd.read_csv('Final_Class1.csv',header=None, skiprows=1)

In [62]:
df2

,0,1,2,3,4
0,0.000056,-0.000035,0.000020,-0.000027,1
1,0.000281,-0.000054,-0.000019,-0.000013,1
2,0.000362,0.000109,-0.000036,0.000024,1
3,-0.000003,-0.000094,0.000016,-0.000019,1
4,-0.000118,-0.000018,0.000042,-0.000023,1
...,...,...,...,...,...
2387,0.000205,-0.000074,0.000041,0.000006,1
2388,0.000223,-0.000002,0.000060,0.000003,1
2389,0.000025,-0.000001,0.000029,0.000025,1
2390,-0.000070,0.000046,0.000015,0.000021,1


In [63]:

# Load the CSV file into a DataFrame
df3 = pd.read_csv('Final_Class2.csv',header=None, skiprows=1)

In [64]:
df3

,0,1,2,3,4
0,-0.000238,0.000008,-1.098566e-05,-3.094375e-06,2
1,-0.000190,-0.000025,-2.928873e-06,4.443215e-06,2
2,-0.000232,-0.000011,-9.544331e-06,1.550101e-06,2
3,-0.000265,0.000013,-1.077376e-05,7.368191e-06,2
4,-0.000257,-0.000013,-6.182824e-07,6.888443e-06,2
...,...,...,...,...,...
2387,-0.000188,-0.000015,5.664910e-06,8.595146e-08,2
2388,-0.000198,0.000017,-2.938213e-06,1.938712e-06,2
2389,-0.000177,-0.000014,1.063117e-05,-2.615482e-06,2
2390,-0.000228,-0.000002,1.678523e-06,-6.784367e-06,2


In [65]:
data1=df1
data2=df2
data3=df3

In [66]:
result = pd.concat([data1,data2,data3],axis=0)
print(result)

             0             1             2             3  4
0    -0.000021 -3.166390e-07 -1.377979e-06  3.863017e-07  0
1    -0.000026  4.598534e-06 -1.489110e-06  6.813670e-07  0
2    -0.000017 -3.372569e-06 -7.059989e-07 -1.842810e-06  0
3    -0.000006  3.581135e-06 -1.727936e-06  3.352294e-08  0
4    -0.000017 -1.060639e-05 -3.921169e-06 -2.589434e-06  0
...        ...           ...           ...           ... ..
2387 -0.000188 -1.522760e-05  5.664910e-06  8.595146e-08  2
2388 -0.000198  1.710231e-05 -2.938213e-06  1.938712e-06  2
2389 -0.000177 -1.435524e-05  1.063117e-05 -2.615482e-06  2
2390 -0.000228 -2.228755e-06  1.678523e-06 -6.784367e-06  2
2391 -0.000117 -3.358709e-05 -2.974637e-06 -6.680293e-06  2

[7176 rows x 5 columns]


In [67]:
np.savetxt("FinalSlicedEMG.csv", result, delimiter=",")

In [68]:
df=result

In [69]:
df

,0,1,2,3,4
0,-0.000021,-3.166390e-07,-1.377979e-06,3.863017e-07,0
1,-0.000026,4.598534e-06,-1.489110e-06,6.813670e-07,0
2,-0.000017,-3.372569e-06,-7.059989e-07,-1.842810e-06,0
3,-0.000006,3.581135e-06,-1.727936e-06,3.352294e-08,0
4,-0.000017,-1.060639e-05,-3.921169e-06,-2.589434e-06,0
...,...,...,...,...,...
2387,-0.000188,-1.522760e-05,5.664910e-06,8.595146e-08,2
2388,-0.000198,1.710231e-05,-2.938213e-06,1.938712e-06,2
2389,-0.000177,-1.435524e-05,1.063117e-05,-2.615482e-06,2
2390,-0.000228,-2.228755e-06,1.678523e-06,-6.784367e-06,2


In [70]:
df.head()

,0,1,2,3,4
0,-0.000021,-3.166390e-07,-1.377979e-06,3.863017e-07,0
1,-0.000026,4.598534e-06,-1.489110e-06,6.813670e-07,0
2,-0.000017,-3.372569e-06,-7.059989e-07,-1.842810e-06,0
3,-0.000006,3.581135e-06,-1.727936e-06,3.352294e-08,0
4,-0.000017,-1.060639e-05,-3.921169e-06,-2.589434e-06,0


In [71]:
df.tail()

,0,1,2,3,4
2387,-0.000188,-0.000015,0.000006,8.595146e-08,2
2388,-0.000198,0.000017,-0.000003,1.938712e-06,2
2389,-0.000177,-0.000014,0.000011,-2.615482e-06,2
2390,-0.000228,-0.000002,0.000002,-6.784367e-06,2
2391,-0.000117,-0.000034,-0.000003,-6.680293e-06,2


In [72]:
#Check for NaN under a single DataFrame column
df.isnull().values.any()

False

In [73]:
# statistical measures about the data
df.describe()

,0,1,2,3,4
count,7.176000e+03,7.176000e+03,7.176000e+03,7.176000e+03,7176.000000
mean,8.262790e-18,-4.495790e-20,-1.764779e-19,1.458535e-20,1.000000
std,2.345268e-04,4.417690e-05,2.354673e-05,1.591760e-05,0.816553
min,-3.144768e-04,-3.739816e-04,-2.562988e-04,-1.608563e-04,0.000000
25%,-1.433272e-04,-1.033662e-05,-6.907375e-06,-3.141855e-06,0.000000
50%,-2.589931e-05,9.684154e-08,-8.415911e-07,-1.364052e-08,1.000000
75%,2.598302e-05,9.273012e-06,4.411235e-06,3.600796e-06,2.000000
max,1.851705e-03,4.540348e-04,3.826513e-04,1.818351e-04,2.000000


In [74]:
X = df.drop(columns=4, axis=1)
Y = df[4]

In [75]:
X

,0,1,2,3
0,-0.000021,-3.166390e-07,-1.377979e-06,3.863017e-07
1,-0.000026,4.598534e-06,-1.489110e-06,6.813670e-07
2,-0.000017,-3.372569e-06,-7.059989e-07,-1.842810e-06
3,-0.000006,3.581135e-06,-1.727936e-06,3.352294e-08
4,-0.000017,-1.060639e-05,-3.921169e-06,-2.589434e-06
...,...,...,...,...
2387,-0.000188,-1.522760e-05,5.664910e-06,8.595146e-08
2388,-0.000198,1.710231e-05,-2.938213e-06,1.938712e-06
2389,-0.000177,-1.435524e-05,1.063117e-05,-2.615482e-06
2390,-0.000228,-2.228755e-06,1.678523e-06,-6.784367e-06


In [76]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
2387    2
2388    2
2389    2
2390    2
2391    2
Name: 4, Length: 7176, dtype: int64

In [77]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,
                                                 test_size=0.3, random_state = 0)
# describes info about train and test set
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", Y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", Y_test.shape)

Number transactions X_train dataset:  (5023, 4)
Number transactions y_train dataset:  (5023,)
Number transactions X_test dataset:  (2153, 4)
Number transactions y_test dataset:  (2153,)


In [78]:
modelRF = RandomForestClassifier()
modelRF.fit(X_train,Y_train)
# accuracy on training data
X_train_prediction = modelRF.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print("Applying Random Forest")
#print('Accuracy on Train data : ', train_data_accuracy)

# print('Presion :',precision_score(X_train_prediction,Y_train))

# print('Recall :',recall_score(X_train_prediction,Y_train))

# print('F1 score : ', f1_score(X_train_prediction,Y_train))

# accuracy on test data
X_test_prediction = modelRF.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy on Test data : ', test_data_accuracy)

print('Precision :', precision_score(Y_test, X_test_prediction, average='weighted'))

print('Recall :', recall_score(Y_test, X_test_prediction, average='weighted'))

print('F1 score : ', f1_score(Y_test, X_test_prediction, average='weighted'))


# get the confusion matrix
conf_mat = confusion_matrix(Y_test, X_test_prediction)
print("Confusion matrix:\n", conf_mat)


Applying Random Forest
Accuracy on Test data :  0.8039944263817929
Precision : 0.8051374354846844
Recall : 0.8039944263817929
F1 score :  0.8038781714975474
Confusion matrix:
 [[689  18   5]
 [ 21 541 159]
 [  4 215 501]]


In [79]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

modelknn = KNeighborsClassifier()
modelknn.fit(X_train, Y_train)

# accuracy on training data
X_train_prediction = modelknn.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('\nResults obtained for the knn')
print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)

print('Presion :', precision_score(Y_train, X_train_prediction, average='weighted'))
print('Recall :', recall_score(Y_train, X_train_prediction, average='weighted'))
print('F1 score : ', f1_score(Y_train, X_train_prediction, average='weighted'))

# Accuracy on test data
X_test_prediction = modelknn.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)

print('Presion :', precision_score(Y_test, X_test_prediction, average='weighted'))
print('Recall :', recall_score(Y_test, X_test_prediction, average='weighted'))
print('F1 score : ', f1_score(Y_test, X_test_prediction, average='weighted'))
# get the confusion matrix
conf_mat = confusion_matrix(Y_test, X_test_prediction)
print("Confusion matrix:\n", conf_mat)



Results obtained for the knn

Results obtained on Training Data
Accuracy on Train data :  0.8385427035636074
Presion : 0.8380264143496834
Recall : 0.8385427035636074
F1 score :  0.8361640264873639

Results obtained on Testing Data
Accuracy on Test data :  0.7807710171853228
Presion : 0.7774087723547712
Recall : 0.7807710171853228
F1 score :  0.7771390130897857
Confusion matrix:
 [[703   4   5]
 [ 55 514 152]
 [ 15 241 464]]


C:\Users\Aditya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Aditya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

In [80]:
modelDT = DecisionTreeClassifier(max_depth=9)
modelDT.fit(X_train,Y_train)



# accuracy on training data
X_train_prediction = modelDT.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction ,Y_train)

print('Accuracy on Train data : ', train_data_accuracy)

print('Presion :',precision_score(X_train_prediction,Y_train,average='weighted'))

print('Recall :',recall_score(X_train_prediction,Y_train,average='weighted'))

print('F1 score : ', f1_score(X_train_prediction,Y_train,average='weighted'))

# accuracy on test data
X_test_prediction = modelRF.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy on Test data : ', test_data_accuracy)

print('Presion :',precision_score(X_test_prediction,Y_test,average='weighted'))
print('Recall :',recall_score(X_test_prediction,Y_test,average='weighted'))

print('F1 score : ', f1_score(X_test_prediction,Y_test,average='weighted'))

Accuracy on Train data :  0.866215409118057
Presion : 0.8765868554857031
Recall : 0.866215409118057
F1 score :  0.8673252494454794
Accuracy on Test data :  0.8039944263817929
Presion : 0.8055889164788238
Recall : 0.8039944263817929
F1 score :  0.8041106812660384
